In [ ]:
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Load pretrained DenseNet model trained on chest X-rays
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()

print("Model loaded successfully ✅")


ModuleNotFoundError: No module named 'torchxrayvision'

In [ ]:
!pip install torchxrayvision --quiet


In [ ]:
!pip install torchxrayvision --quiet


In [ ]:
import torchxrayvision as xrv
print("TorchXRayVision working ✅")


ModuleNotFoundError: No module named 'torchxrayvision'

In [ ]:
import torch

model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()

print("Model loaded successfully ✅")


In [ ]:
from google.colab import files
from PIL import Image
import torchvision.transforms as transforms

uploaded = files.upload()
img_path = list(uploaded.keys())[0]

# Load X-ray as grayscale
img = Image.open(img_path).convert("L")

# Transform for model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

img = transform(img).unsqueeze(0)

print("Image loaded and ready ✅")


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import torch
import torchxrayvision as xrv

with torch.no_grad():
    preds = model(img)

diseases = xrv.datasets.default_pathologies

results = {}
for i, d in enumerate(diseases):
    results[d] = float(preds[0][i])

# Sort by probability
sorted_results = dict(sorted(results.items(), key=lambda x: x[1], reverse=True))

print("\nTop predicted findings:\n")
for k, v in list(sorted_results.items())[:10]:
    print(f"{k}: {v:.3f}")


In [ ]:
def generate_radiology_report(preds_dict, clinical_text, threshold=0.4):
    findings = []
    impression = []

    for disease, prob in preds_dict.items():
        if prob > threshold:
            if disease == "Effusion":
                findings.append("Blunting of the costophrenic angle suggesting pleural effusion.")
                impression.append("Pleural effusion likely present.")
            elif disease == "Cardiomegaly":
                findings.append("Cardiac silhouette appears enlarged.")
                impression.append("Findings consistent with cardiomegaly.")
            elif disease == "Atelectasis":
                findings.append("Linear opacities suggesting atelectasis.")
                impression.append("Possible atelectasis.")
            elif disease == "Pneumonia":
                findings.append("Patchy airspace opacity suspicious for pneumonia.")
                impression.append("Pneumonia cannot be excluded.")
            elif disease == "Consolidation":
                findings.append("Focal consolidation noted in lung field.")
                impression.append("Consolidation likely.")
            elif disease == "Edema":
                findings.append("Diffuse interstitial markings suggesting pulmonary edema.")
                impression.append("Pulmonary edema suspected.")

    if len(findings) == 0:
        findings.append("No acute cardiopulmonary abnormality detected.")
        impression.append("Study appears within normal limits.")

    report = f"""
CLINICAL INDICATION:
{clinical_text}

FINDINGS:
{" ".join(findings)}

IMPRESSION:
{" ".join(impression)}
"""
    return report


In [ ]:
# Use top predictions dictionary (from earlier cell)
clinical_text = "65 year old patient with cough and shortness of breath"

report = generate_radiology_report(sorted_results, clinical_text)

print("\n===== GENERATED RADIOLOGY REPORT =====\n")
print(report)


In [ ]:
from google.colab import files
from PIL import Image
import torchvision.transforms as transforms

uploaded = files.upload()
img_path = list(uploaded.keys())[0]

img = Image.open(img_path).convert("L")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

img = transform(img).unsqueeze(0)

print("New image loaded ✅")


In [ ]:
import torch
import torchxrayvision as xrv

with torch.no_grad():
    preds = model(img)

diseases = xrv.datasets.default_pathologies

results = {}
for i, d in enumerate(diseases):
    results[d] = float(preds[0][i])

sorted_results = dict(sorted(results.items(), key=lambda x: x[1], reverse=True))

print("\nTop predicted findings:\n")
for k, v in list(sorted_results.items())[:10]:
    print(f"{k}: {v:.3f}")


In [ ]:
clinical_text = input("Enter clinical indication: ")

report = generate_radiology_report(sorted_results, clinical_text)

print("\n===== GENERATED RADIOLOGY REPORT =====\n")
print(report)


In [ ]:
Get real chest X-ray sample set

Add confidence scores

Show heatmap (where model looked)

Improve medical wording

Make automatic batch testing

In [ ]:
!mkdir xrays
!wget -q https://raw.githubusercontent.com/mlmed/torchxrayvision/master/tests/assets/nih_sample.png -O xrays/xray1.png
!wget -q https://raw.githubusercontent.com/mlmed/torchxrayvision/master/tests/assets/nih_sample2.png -O xrays/xray2.png
!wget -q https://raw.githubusercontent.com/mlmed/torchxrayvision/master/tests/assets/nih_sample3.png -O xrays/xray3.png

print("Sample X-rays downloaded ✅")


In [ ]:
def show_predictions_with_confidence(preds, diseases, topk=8):
    scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
    scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

    print("\n=== Disease Confidence Scores ===\n")
    for k, v in list(scores.items())[:topk]:
        print(f"{k:20s} : {v:.3f}")

    return scores


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def generate_heatmap(model, img):
    img.requires_grad = True
    output = model(img)
    score = output.max()
    score.backward()

    gradients = img.grad.data[0].numpy()[0]
    heatmap = np.maximum(gradients, 0)
    heatmap = heatmap / (heatmap.max() + 1e-8)

    plt.imshow(heatmap, cmap='jet')
    plt.title("Model Attention Heatmap")
    plt.axis('off')
    plt.show()


In [ ]:
def generate_better_report(preds_dict, clinical_text, threshold=0.4):
    findings = []
    impression = []

    for disease, prob in preds_dict.items():
        if prob > threshold:
            severity = "mild"
            if prob > 0.7:
                severity = "moderate"
            if prob > 0.85:
                severity = "significant"

            findings.append(f"{severity.capitalize()} radiographic features of {disease.lower()} observed.")
            impression.append(f"{disease} is {severity}ly likely.")

    if not findings:
        findings.append("Lungs are clear. No focal consolidation, pleural effusion, or pneumothorax.")
        impression.append("No acute cardiopulmonary abnormality.")

    report = f"""
CLINICAL INDICATION:
{clinical_text}

FINDINGS:
{" ".join(findings)}

IMPRESSION:
{" ".join(impression)}
"""
    return report


In [ ]:
import os
from PIL import Image
import torchvision.transforms as transforms
import torchxrayvision as xrv

def batch_test(folder="xrays"):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    diseases = xrv.datasets.default_pathologies

    for file in os.listdir(folder):
        path = os.path.join(folder, file)

        img = Image.open(path).convert("L")
        img_t = transform(img).unsqueeze(0)

        with torch.no_grad():
            preds = model(img_t)

        scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
        scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

        report = generate_better_report(scores, "Automated batch test")

        print("\n==============================")
        print("Image:", file)
        print("==============================")
        print(report)

batch_test("xrays")


In [ ]:
!rm -rf xrays


In [ ]:
!mkdir xrays
!wget -O xrays/xray1.png https://github.com/mlmed/torchxrayvision/raw/master/tests/assets/nih_sample.png
!wget -O xrays/xray2.png https://github.com/mlmed/torchxrayvision/raw/master/tests/assets/nih_sample2.png
!wget -O xrays/xray3.png https://github.com/mlmed/torchxrayvision/raw/master/tests/assets/nih_sample3.png

print("Real X-ray images downloaded successfully ✅")


In [ ]:
batch_test("xrays")


In [ ]:
!rm -rf xrays


In [ ]:
!mkdir xrays

!wget -q --show-progress https://github.com/mlmed/torchxrayvision/raw/master/tests/assets/nih_sample.png?raw=true -O xrays/xray1.png
!wget -q --show-progress https://github.com/mlmed/torchxrayvision/raw/master/tests/assets/nih_sample2.png?raw=true -O xrays/xray2.png
!wget -q --show-progress https://github.com/mlmed/torchxrayvision/raw/master/tests/assets/nih_sample3.png?raw=true -O xrays/xray3.png

print("X-ray images downloaded correctly ✅")


In [ ]:
from PIL import Image
display(Image.open("xrays/xray1.png"))


In [ ]:
!rm -rf xrays
!mkdir xrays


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import os, shutil

for f in uploaded.keys():
    shutil.move(f, f"xrays/{f}")

print("Images moved to xrays/ folder ✅")


In [ ]:
from PIL import Image
import os

for f in os.listdir("xrays"):
    display(Image.open(f"xrays/{f}"))


In [ ]:
batch_test("xrays")


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def gradcam(model, img):
    model.eval()
    img = img.clone().detach()
    img.requires_grad_(True)

    output = model(img)
    score = output.max()
    score.backward()

    gradients = img.grad[0].cpu().numpy()[0]
    heatmap = np.maximum(gradients, 0)
    heatmap /= (heatmap.max() + 1e-8)

    plt.imshow(heatmap, cmap='jet')


In [ ]:
gradcam(model, img_t)


In [ ]:
def generate_realistic_report(preds_dict, clinical_text, threshold=0.45):
    findings = []
    impression = []
    negatives = []

    important = ["Effusion", "Cardiomegaly", "Pneumonia", "Consolidation", "Edema", "Atelectasis"]

    for disease, prob in preds_dict.items():
        if disease not in important:
            continue

        if prob > threshold:
            if prob > 0.8:
                severity = "significant"
            elif prob > 0.6:
                severity = "moderate"
            else:
                severity = "mild"

            findings.append(f"{severity.capitalize()} radiographic evidence of {disease.lower()}.")
            impression.append(f"{disease} is {severity}ly likely.")
        else:
            negatives.append(disease.lower())

    if not findings:
        findings.append("Lungs are clear without focal consolidation, effusion, or pneumothorax.")
        impression.append("No acute cardiopulmonary abnormality.")

    report = f"""
CLINICAL INDICATION:
{clinical_text}

FINDINGS:
{" ".join(findings)}

NEGATIVE FINDINGS:
No clear radiographic evidence of {", ".join(negatives[:5])}.

IMPRESSION:
{" ".join(impression)}
"""
    return report


In [ ]:
def generate_realistic_report(preds_dict, clinical_text, threshold=0.45):
    findings = []
    impression = []
    negatives = []

    important = ["Effusion", "Cardiomegaly", "Pneumonia", "Consolidation", "Edema", "Atelectasis"]

    for disease, prob in preds_dict.items():
        if disease not in important:
            continue

        if prob > threshold:
            if prob > 0.8:
                severity = "significant"
            elif prob > 0.6:
                severity = "moderate"
            else:
                severity = "mild"

            findings.append(f"{severity.capitalize()} radiographic evidence of {disease.lower()}.")
            impression.append(f"{disease} is {severity}ly likely.")
        else:
            negatives.append(disease.lower())

    if not findings:
        findings.append("Lungs are clear without focal consolidation, effusion, or pneumothorax.")
        impression.append("No acute cardiopulmonary abnormality.")

    report = f"""
CLINICAL INDICATION:
{clinical_text}

FINDINGS:
{" ".join(findings)}

NEGATIVE FINDINGS:
No clear radiographic evidence of {", ".join(negatives[:5])}.

IMPRESSION:
{" ".join(impression)}
"""
    return report


In [ ]:
from google.colab import files
uploaded = files.upload()

img_path = list(uploaded.keys())[0]
print("Uploaded:", img_path)


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

img = Image.open(img_path).convert("L")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

img_t = transform(img).unsqueeze(0)
print("Image processed ✅")


In [ ]:
with torch.no_grad():
    preds = model(img_t)

scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

print("\nTop Predictions:\n")
for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
clinical_text = input("Enter clinical indication: ")

report = generate_realistic_report(scores, clinical_text)
print("\n===== RADIOLOGY REPORT =====\n")
print(report)


In [ ]:
gradcam(model, img_t)



In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

def real_gradcam(model, img):
    model.eval()

    # Get last conv layer
    target_layer = model.features[-1]

    activations = []
    gradients = []

    def forward_hook(module, inp, out):
        activations.append(out)

    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0])

    # Register hooks
    fh = target_layer.register_forward_hook(forward_hook)
    bh = target_layer.register_backward_hook(backward_hook)

    # Forward
    output = model(img)
    class_idx = output.argmax()
    score = output[0, class_idx]

    # Backward
    model.zero_grad()
    score.backward()

    # Get data
    grads = gradients[0].detach().cpu().numpy()[0]
    acts = activations[0].detach().cpu().numpy()[0]

    weights = np.mean(grads, axis=(1, 2))
    cam = np.zeros(acts.shape[1:], dtype=np.float32)

    for i, w in enumerate(weights):
        cam += w * acts[i]

    cam = np.maximum(cam, 0)
    cam = cam / (cam.max() + 1e-8)

    # Resize to image size
    cam = cv2.resize(cam, (224, 224))

    # Convert original image
    img_np = img.detach().cpu().numpy()[0][0]

    # Overlay heatmap
    plt.figure(figsize=(5,5))
    plt.imshow(img_np, cmap="gray")
    plt.imshow(cam, cmap="jet", alpha=0.4)
    plt.title("Grad-CAM (Model Focus)")
    plt.axis("off")
    plt.show()

    # Remove hooks
    fh.remove()
    bh.remove()


In [ ]:
real_gradcam(model, img_t)


In [ ]:
import json

kaggle_dict = {
    "username": "krishiket",
    "key": "KGAT_d18afec69c0e78ca315bfd00ee871f08"
}

with open("kaggle.json", "w") as f:
    json.dump(kaggle_dict, f)

print("kaggle.json created manually ✅")


In [ ]:
 !mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle setup complete ✅")


In [ ]:
!pip install -q kaggle


In [ ]:
!kaggle datasets download -d raddar/chest-xrays-indiana-university


In [ ]:
!kaggle datasets download -d raddar/chest-xrays-indiana-university


In [ ]:
!pip install -q kagglehub


In [ ]:
import kagglehub

path = kagglehub.dataset_download("raddar/chest-xrays-indiana-university")

print("Dataset downloaded to:", path)


In [ ]:
import os
print(os.listdir(path))


In [ ]:
import os

print(os.listdir(path))



In [ ]:
for root, dirs, files in os.walk(path):
    for f in files[:5]:
        print(os.path.join(root, f))
    break


In [ ]:
for root, dirs, files in os.walk(path):
    if len(files) > 0:
        print("FOLDER:", root)
        print("Sample files:", files[:5])
        break


In [ ]:
import os

img_root = os.path.join(path, "images")
print("Inside images folder:", os.listdir(img_root)[:5])


In [ ]:
img_folder = os.path.join(img_root, "images_normalized")
print("Sample images:", os.listdir(img_folder)[:5])


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

img_path = os.path.join(img_folder, os.listdir(img_folder)[0])
print("Loading:", img_path)

img = Image.open(img_path).convert("L")

plt.imshow(img, cmap="gray")
plt.axis("off")
plt.title("IU Chest X-ray Sample")


In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

img_t = transform(img).unsqueeze(0)

with torch.no_grad():
    preds = model(img_t)

scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image


In [ ]:
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()

diseases = xrv.datasets.default_pathologies
print("Model ready ✅")


In [ ]:
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image

# Load model
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()

# Disease labels
diseases = xrv.datasets.default_pathologies

print("Model ready ✅")


In [ ]:
!pip install -q torchxrayvision


In [ ]:
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image

model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()

diseases = xrv.datasets.default_pathologies

print("Model ready ✅")


In [ ]:
img_t = transform(img).unsqueeze(0)

with torch.no_grad():
    preds = model(img_t)

scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
report = generate_realistic_report(scores, "Patient from IU Chest X-ray dataset")

print("\n===== AI RADIOLOGY REPORT (IU DATASET) =====\n")
print(report)


In [ ]:
report = generate_realistic_report(scores, "Patient from IU Chest X-ray dataset")

print("\n===== AI RADIOLOGY REPORT (IU DATASET) =====\n")
print(report)



In [ ]:
def generate_realistic_report(preds_dict, clinical_text, threshold=0.45):
    findings = []
    impression = []
    negatives = []

    important = ["Effusion", "Cardiomegaly", "Pneumonia", "Consolidation", "Edema", "Atelectasis"]

    for disease, prob in preds_dict.items():
        if disease not in important:
            continue

        if prob > threshold:
            if prob > 0.8:
                severity = "significant"
            elif prob > 0.6:
                severity = "moderate"
            else:
                severity = "mild"

            findings.append(f"{severity.capitalize()} radiographic evidence of {disease.lower()}.")
            impression.append(f"{disease} is {severity}ly likely.")
        else:
            negatives.append(disease.lower())

    if not findings:
        findings.append("Lungs are clear without focal consolidation, effusion, or pneumothorax.")
        impression.append("No acute cardiopulmonary abnormality.")

    report = f"""
CLINICAL INDICATION:
{clinical_text}

FINDINGS:
{" ".join(findings)}

NEGATIVE FINDINGS:
No clear radiographic evidence of {", ".join(negatives[:5])}.

IMPRESSION:
{" ".join(impression)}
"""
    return report


In [ ]:
report = generate_realistic_report(scores, "Patient from IU Chest X-ray dataset")

print("\n===== AI RADIOLOGY REPORT (IU DATASET) =====\n")
print(report)


In [ ]:
import pandas as pd
import os

reports_path = os.path.join(path, "indiana_reports.csv")
df_reports = pd.read_csv(reports_path)

print("Columns:", df_reports.columns)
print("\nSample row:\n")
print(df_reports.iloc[0])


In [ ]:
print("\n===== REAL RADIOLOGIST REPORT =====\n")

print("FINDINGS:\n")
print(df_reports.iloc[0]["findings"])

print("\nIMPRESSION:\n")
print(df_reports.iloc[0]["impression"])


In [ ]:
1. Make AI learn writing style from real IU reports (BIG improvement)
2. Improve disease reasoning (better medical accuracy)
3. Integrate your Cognitive Modules (PRO-FA, MIX-MLP, RCTA)

In [ ]:
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^a-z0-9., ]", "", text)
    return text

df_reports["findings_clean"] = df_reports["findings"].apply(clean_text)
df_reports["impression_clean"] = df_reports["impression"].apply(clean_text)

print("Cleaned ✔")


In [ ]:
phrase_bank_findings = []
phrase_bank_impression = []

for i in range(min(300, len(df_reports))):
    f = df_reports.iloc[i]["findings_clean"]
    im = df_reports.iloc[i]["impression_clean"]

    if len(f) > 30:
        phrase_bank_findings.append(f)
    if len(im) > 20:
        phrase_bank_impression.append(im)

print("Phrase bank ready ✔")


In [ ]:
import random

def generate_learned_report(preds_dict, clinical_text, threshold=0.45):
    findings = []
    impression = []

    important = ["Effusion", "Cardiomegaly", "Pneumonia", "Consolidation", "Edema", "Atelectasis"]

    for disease, prob in preds_dict.items():
        if disease not in important:
            continue

        if prob > threshold:
            severity = "mild"
            if prob > 0.7:
                severity = "moderate"
            if prob > 0.85:
                severity = "severe"

            findings.append(f"{severity} features of {disease.lower()} noted.")
            impression.append(f"{disease.lower()} likely.")

    if not findings:
        findings.append("lungs are clear without acute cardiopulmonary abnormality.")
        impression.append("no acute abnormality detected.")

    # Add real style sentence
    if len(phrase_bank_findings) > 0:
        findings.append(random.choice(phrase_bank_findings))
    if len(phrase_bank_impression) > 0:
        impression.append(random.choice(phrase_bank_impression))

    report = f"""
CLINICAL INDICATION:
{clinical_text}

FINDINGS:
{" ".join(findings)}

IMPRESSION:
{" ".join(impression)}
"""
    return report


In [ ]:
def improve_reasoning(scores):
    # Example medical reasoning rules

    if scores.get("Edema", 0) > 0.6 and scores.get("Cardiomegaly", 0) > 0.5:
        scores["Heart Failure Pattern"] = 0.9

    if scores.get("Consolidation", 0) > 0.5 and scores.get("Pneumonia", 0) > 0.4:
        scores["Infectious Pattern"] = 0.85

    if scores.get("Effusion", 0) > 0.6 and scores.get("Atelectasis", 0) > 0.4:
        scores["Pleural Disease Pattern"] = 0.8

    return scores


In [ ]:
scores = improve_reasoning(scores)


In [ ]:
def pro_fa_simulation(img):
    organ_feature = img.mean().item()
    region_feature = img[:, :, 80:150, 80:150].mean().item()
    pixel_feature = img.std().item()
    return [organ_feature, region_feature, pixel_feature]


In [ ]:
def mix_mlp_simulation(scores):
    # Combine disease features
    confidence = sum(scores.values()) / len(scores)
    return confidence


In [ ]:
def rcta_simulation(img, clinical_text, scores):
    visual = pro_fa_simulation(img)
    diagnosis_conf = mix_mlp_simulation(scores)

    context_score = len(clinical_text) * 0.01
    final_conf = (sum(visual) + diagnosis_conf + context_score) / 3

    return final_conf


In [ ]:
final_confidence = rcta_simulation(img_t, "Patient with cough", scores)
print("Cognitive confidence:", final_confidence)


In [ ]:
scores = improve_reasoning(scores)

report = generate_learned_report(scores, "Patient with cough and fever")
print(report)


In [ ]:
!pip install -q streamlit pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image

# Load model once
@st.cache_resource
def load_model():
    model = xrv.models.DenseNet(weights="densenet121-res224-all")
    model.eval()
    diseases = xrv.datasets.default_pathologies
    return model, diseases

model, diseases = load_model()

# Simple report generator
def generate_report(scores):
    findings = []
    impression = []

    for d, p in scores.items():
        if p > 0.5:
            findings.append(f"{d} detected with confidence {p:.2f}")
            impression.append(f"Possible {d}")

    if not findings:
        findings.append("No acute abnormality detected.")
        impression.append("Normal study.")

    return f"""
FINDINGS:
{' '.join(findings)}

IMPRESSION:
{' '.join(impression)}
"""

st.title("🩺 AI Cognitive Radiology Demo")

uploaded_file = st.file_uploader("Upload Chest X-ray", type=["png", "jpg", "jpeg"])

if uploaded_file:
    img = Image.open(uploaded_file).convert("L")
    st.image(img, caption="Uploaded X-ray", use_column_width=True)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    img_t = transform(img).unsqueeze(0)

    with torch.no_grad():
        preds = model(img_t)

    scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
    scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

    st.subheader("Top Predictions")
    for k, v in list(scores.items())[:5]:
        st.write(f"{k}: {v:.3f}")

    report = generate_report(scores)

    st.subheader("AI Radiology Report")
    st.text(report)


In [ ]:
from pyngrok import ngrok
import subprocess
import time

# Start Streamlit
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])

time.sleep(5)

public_url = ngrok.connect(8501)
print("OPEN THIS LINK 👉", public_url)


In [ ]:
!pkill -f streamlit
!pkill -f ngrok


In [ ]:
!npm install -g localtunnel


In [ ]:
import subprocess
import time

# Start Streamlit
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])

time.sleep(5)

# Create public tunnel WITHOUT ngrok
print("Starting public link...")
tunnel = subprocess.Popen(["lt", "--port", "8501"], stdout=subprocess.PIPE)

time.sleep(5)


In [ ]:
!curl ipv4.icanhazip.com


In [ ]:
!lt --port 8501


In [ ]:
!pkill -f streamlit
!pkill -f lt
!pkill -f python


In [ ]:
!pkill -f streamlit
!pkill -f lt
!pkill -f python


In [ ]:
!pip install -q streamlit torch torchvision torchxrayvision



In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torchxrayvision as xrv
import torchvision.transforms as transforms
from PIL import Image

st.set_page_config(page_title="AI Cognitive Radiology", layout="centered")

# Load model once
@st.cache_resource
def load_model():
    model = xrv.models.DenseNet(weights="densenet121-res224-all")
    model.eval()
    diseases = xrv.datasets.default_pathologies
    return model, diseases

model, diseases = load_model()

def generate_report(scores, clinical_text):
    findings = []
    impression = []

    for d, p in scores.items():
        if p > 0.5:
            findings.append(f"{d} detected (confidence {p:.2f})")
            impression.append(f"Possible {d}")

    if not findings:
        findings.append("No acute cardiopulmonary abnormality.")
        impression.append("Normal study.")

    return f"""
CLINICAL INDICATION:
{clinical_text}

FINDINGS:
{' '.join(findings)}

IMPRESSION:
{' '.join(impression)}
"""

st.title("🩺 AI Cognitive Radiology Demo")

clinical_text = st.text_input(
    "Clinical Indication",
    "Patient with cough and fever"
)

uploaded_file = st.file_uploader("Upload Chest X-ray", type=["png", "jpg", "jpeg"])

if uploaded_file:
    img = Image.open(uploaded_file).convert("L")
    st.image(img, caption="Uploaded X-ray", use_column_width=True)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    img_t = transform(img).unsqueeze(0)

    with torch.no_grad():
        preds = model(img_t)

    scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
    scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

    st.subheader("Top Predictions")
    for k, v in list(scores.items())[:5]:
        st.write(f"{k}: {v:.3f}")

    confidence = sum(scores.values()) / len(scores)
    st.metric("Cognitive Confidence", f"{confidence:.2f}")

    report = generate_report(scores, clinical_text)

    st.subheader("AI Radiology Report")
    st.text(report)


In [ ]:
import subprocess, time
subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])
time.sleep(6)
print("Streamlit started")



In [ ]:
!lt --port 8501


In [ ]:
!npx localtunnel --port 8501


In [ ]:
!pip install -q flask flask-ngrok torch torchvision torchxrayvision pillow


In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return "<h1>Flask is working in Colab 🚀</h1>"

app.run()


In [ ]:
!pip install -q gradio


In [ ]:
import gradio as gr

def test():
    return "AI Cognitive Radiology is running 🚀"

demo = gr.Interface(
    fn=test,
    inputs=[],
    outputs="text",
    title="AI Cognitive Radiology"
)

demo.launch(share=True)


In [ ]:
import gradio as gr
import torch
from PIL import Image
import torchvision.transforms as transforms
import torchxrayvision as xrv

# Load model
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()
diseases = xrv.datasets.default_pathologies

# Image transform
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

def predict_xray(img):
    img = img.convert("RGB")
    img_t = transform(img).unsqueeze(0)

    with torch.no_grad():
        preds = model(img_t)[0]

    scores = {diseases[i]: float(preds[i]) for i in range(len(diseases))}
    scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

    top5 = "\n".join([f"{k}: {v:.3f}" for k,v in list(scores.items())[:5]])
    return top5

demo = gr.Interface(
    fn=predict_xray,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="AI Cognitive Radiology — X-ray Analyzer"
)

demo.launch(share=True)


In [ ]:
import gradio as gr
import torch
from PIL import Image
import torchvision.transforms as transforms
import torchxrayvision as xrv

# Load model
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()
diseases = xrv.datasets.default_pathologies

# Image transform
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

def predict_xray(img):
    img = img.convert("RGB")
    img_t = transform(img).unsqueeze(0)

    with torch.no_grad():
        preds = model(img_t)[0]

    scores = {diseases[i]: float(preds[i]) for i in range(len(diseases))}
    scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

    top5 = "\n".join([f"{k}: {v:.3f}" for k,v in list(scores.items())[:5]])
    return top5

demo = gr.Interface(
    fn=predict_xray,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="AI Cognitive Radiology — X-ray Analyzer"
)

demo.launch(share=True)


In [ ]:
pip install streamlit torch torchvision pillow torchxrayvision


In [ ]:
import time
import sys

def animated_generate_process(text="Generating Cognitive Report"):
    frames = ["⠋","⠙","⠹","⠸","⠼","⠴","⠦","⠧","⠇","⠏"]
    for i in range(30):
        frame = frames[i % len(frames)]
        sys.stdout.write(f"\r{frame} {text}...")
        sys.stdout.flush()
        time.sleep(0.08)
    print("\r✔ Generation Complete!          ")


In [ ]:
def pulse_effect():
    for i in range(3):
        print("⚡ Cognitive Engine Activating" + "." * (i+1))
        time.sleep(0.3)

pulse_effect()


In [ ]:
!pip install -q streamlit pyngrok torch torchvision pillow torchxrayvision


In [ ]:
%%writefile app.py
import streamlit as st
import time
import torchxrayvision as xrv

st.set_page_config(page_title="AI Cognitive Radiology")

# ---------- YOUR LOADER CSS ----------
st.markdown("""
<style>
.main-container {
  display:flex;
  justify-content:center;
  align-items:center;
  height:350px;
}

.loader {
  width:320px;
}

.skeleton {
  fill:#2d2d2d;
  animation:pulse 1.8s ease-in-out infinite;
}

@keyframes pulse {
  0%{fill:#2d2d2d;}
  50%{fill:#505050;}
  100%{fill:#2d2d2d;}
}
</style>
""", unsafe_allow_html=True)


def show_startup_loader():
    return """
    <div class="main-container">
        <svg class="loader" viewBox="0 0 200 100">
            <rect x="10" y="20" width="180" height="60" class="skeleton"/>
            <text x="50%" y="55%" text-anchor="middle" fill="#e4e4e4" font-size="13">
                Initializing Cognitive Radiology Engine...
            </text>
        </svg>
    </div>
    """


# ---------- SHOW LOADER ----------
placeholder = st.empty()
placeholder.markdown(show_startup_loader(), unsafe_allow_html=True)

# ---------- LOAD MODEL (simulate) ----------
@st.cache_resource
def load_model():
    time.sleep(3)  # simulate loading
    model = xrv.models.DenseNet(weights="densenet121-res224-all")
    return model

model = load_model()

# ---------- REMOVE LOADER ----------
placeholder.empty()

st.success("AI Cognitive Radiology Engine Ready 🚀")
st.write("Now your main app will start here...")


In [ ]:
from pyngrok import ngrok
import subprocess, time

# start streamlit
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])
time.sleep(5)

# create public link
public_url = ngrok.connect(8501)
print("OPEN THIS LINK 👉", public_url)


In [ ]:
!pip install -q gradio torch torchvision pillow torchxrayvision


In [ ]:
import gradio as gr
import time
import torchxrayvision as xrv

# ---------- LOAD MODEL ----------
def load_model():
    time.sleep(2)   # simulate startup loading
    model = xrv.models.DenseNet(weights="densenet121-res224-all")
    model.eval()
    return model

model = load_model()


# ---------- REPORT GENERATION ----------
def generate_report(image):
    time.sleep(3)   # simulate AI thinking

    return """
FINDINGS:
No acute cardiopulmonary abnormality. Lungs appear clear.

IMPRESSION:
Normal chest radiograph.
"""


# ---------- UI ----------
with gr.Blocks(title="AI Cognitive Radiology") as app:

    gr.Markdown("# 🧠 AI Cognitive Radiology Engine")

    image_input = gr.Image(type="pil", label="Upload Chest X-ray")

    generate_btn = gr.Button("Generate Report")

    output_text = gr.Textbox(label="Radiology Report", lines=10)

    generate_btn.click(generate_report, inputs=image_input, outputs=output_text)


# ---------- LAUNCH ----------
app.launch(share=True)


In [ ]:
pip install matplotlib opencv-python


In [ ]:
import cv2
import matplotlib.pyplot as plt


In [ ]:
def generate_heatmap(model, img_tensor):
    model.eval()

    # get last conv layer
    target_layer = model.features[-1]

    gradients = []
    activations = []

    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0])

    def forward_hook(module, inp, out):
        activations.append(out)

    handle_fw = target_layer.register_forward_hook(forward_hook)
    handle_bw = target_layer.register_backward_hook(backward_hook)

    output = model(img_tensor)
    pred_class = output.argmax()

    model.zero_grad()
    output[0, pred_class].backward()

    grads = gradients[0].detach().cpu().numpy()[0]
    acts = activations[0].detach().cpu().numpy()[0]

    weights = grads.mean(axis=(1, 2))

    cam = np.zeros(acts.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * acts[i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (img_tensor.shape[3], img_tensor.shape[2]))
    cam = cam - cam.min()
    cam = cam / (cam.max() + 1e-8)

    handle_fw.remove()
    handle_bw.remove()

    return cam


In [ ]:
# -------- HEATMAP --------
cam = generate_heatmap(model, img)

st.subheader("AI Attention Heatmap")

heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)

orig = np.array(image.resize((heatmap.shape[1], heatmap.shape[0])))
if len(orig.shape) == 2:
    orig = cv2.cvtColor(orig, cv2.COLOR_GRAY2BGR)

overlay = cv2.addWeighted(orig, 0.6, heatmap, 0.4, 0)

st.image(overlay, caption="Where AI looked", use_column_width=True)


In [ ]:

def refine_disease_reasoning(scores):
    # simulate medical co-occurrence reasoning
    adjusted = scores.copy()

    # Example clinical logic
    if adjusted.get("Pneumonia", 0) > 0.4:
        adjusted["Consolidation"] = max(adjusted.get("Consolidation", 0), 0.35)

    if adjusted.get("Edema", 0) > 0.4:
        adjusted["Cardiomegaly"] = max(adjusted.get("Cardiomegaly", 0), 0.3)

    if adjusted.get("Pleural Effusion", 0) > 0.5:
        adjusted["Atelectasis"] = max(adjusted.get("Atelectasis", 0), 0.25)

    # Normalize
    total = sum(adjusted.values()) + 1e-8
    for k in adjusted:
        adjusted[k] = adjusted[k] / total

    return adjusted


In [ ]:
scores = refine_disease_reasoning(scores)


In [ ]:
NameError: name 'scores' is not defined


In [ ]:
img_t = transform(img).unsqueeze(0)

with torch.no_grad():
    preds = model(img_t)

scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

print("Raw Scores:")
for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
scores = refine_disease_reasoning(scores)

print("\nRefined Scores:")
for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [ ]:
img_t = transform(img).unsqueeze(0)

with torch.no_grad():
    preds = model(img_t)

scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

print("Raw Scores:")
for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
# Imports
import torch
import torchvision.transforms as transforms
from PIL import Image

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [ ]:
img_t = transform(img).unsqueeze(0)

with torch.no_grad():
    preds = model(img_t)

scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

print("Raw Scores:")
for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
from PIL import Image

# Put your test X-ray path here
image_path = "/root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2/images/CXR1_1_IM-0001-3001.png"

img = Image.open(image_path).convert("RGB")
img


In [ ]:
from PIL import Image

# Put your test X-ray path here
image_path = "/root/.cache/kagglehub/datasets/raddar/chest-xrays-indiana-university/versions/2/images/CXR1_1_IM-0001-3001.png"

img = Image.open(image_path).convert("RGB")
img


In [ ]:
import os

for root, dirs, files in os.walk(path + "/images"):
    for f in files[:20]:
        print(os.path.join(root, f))
    break


In [ ]:
import kagglehub

path = kagglehub.dataset_download("raddar/chest-xrays-indiana-university")
print("Dataset path:", path)


In [ ]:
import os

for root, dirs, files in os.walk(path + "/images"):
    for f in files[:20]:
        print(os.path.join(root, f))
    break


In [ ]:
import os
print(os.listdir(path))


In [ ]:
for root, dirs, files in os.walk(path + "/images"):
    for f in files[:1]:
        image_path = os.path.join(root, f)
        print("Using image:", image_path)
    break


In [ ]:
from PIL import Image
img = Image.open(image_path).convert("RGB")
img


In [ ]:
import os

# search ANY png image inside dataset
image_path = None
for root, dirs, files in os.walk(path):
    for f in files:
        if f.endswith(".png"):
            image_path = os.path.join(root, f)
            break
    if image_path:
        break

print("Image found:", image_path)


In [ ]:
from PIL import Image

img = Image.open(image_path).convert("RGB")
display(img)


In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

img_t = transform(img).unsqueeze(0)

with torch.no_grad():
    preds = model(img_t)

scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
import numpy as np
import torchxrayvision as xrv

# ---- Convert to grayscale numpy ----
img_np = np.array(img.convert("L"))  # force grayscale

# ---- Normalize correctly for torchxrayvision ----
img_np = xrv.datasets.normalize(img_np, 255)

# ---- Convert to tensor shape (1,1,H,W) ----
img_t = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).float()

# ---- Resize to 224 ----
img_t = torch.nn.functional.interpolate(img_t, size=(224,224), mode="bilinear", align_corners=False)

# ---- Predict ----
with torch.no_grad():
    preds = model(img_t)

# ---- Scores ----
scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

print("Top Predictions:")
for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
import torchxrayvision as xrv
diseases = xrv.datasets.default_pathologies
print("Diseases loaded:", len(diseases))


In [ ]:
# ---- Convert to grayscale numpy ----
img_np = np.array(img.convert("L"))

# ---- Normalize correctly ----
img_np = xrv.datasets.normalize(img_np, 255)

# ---- Tensor shape (1,1,H,W) ----
img_t = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).float()

# ---- Resize ----
img_t = torch.nn.functional.interpolate(img_t, size=(224,224), mode="bilinear", align_corners=False)

# ---- Predict ----
with torch.no_grad():
    preds = model(img_t)

# ---- Scores ----
scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

print("Top Predictions:")
for k, v in list(scores.items())[:8]:
    print(f"{k:20s}: {v:.3f}")


In [ ]:
import numpy as np
import cv2
import torch

def generate_heatmap(model, img_tensor):
    model.eval()

    gradients = []
    activations = []

    # last conv layer of DenseNet
    target_layer = model.features[-1]

    def forward_hook(module, inp, out):
        activations.append(out)

    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0])

    h1 = target_layer.register_forward_hook(forward_hook)
    h2 = target_layer.register_backward_hook(backward_hook)

    output = model(img_tensor)
    pred_class = output.argmax()

    model.zero_grad()
    output[0, pred_class].backward()

    grads = gradients[0].detach().cpu().numpy()[0]
    acts = activations[0].detach().cpu().numpy()[0]

    weights = grads.mean(axis=(1,2))

    cam = np.zeros(acts.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * acts[i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (img_tensor.shape[3], img_tensor.shape[2]))
    cam = cam - cam.min()
    cam = cam / (cam.max() + 1e-8)

    h1.remove()
    h2.remove()

    return cam


In [ ]:
cam = generate_heatmap(model, img_t)
print("Heatmap generated")


In [ ]:
# convert heatmap to color
heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)

# original grayscale image resized
orig = np.array(img.convert("L").resize((224,224)))
orig = cv2.cvtColor(orig, cv2.COLOR_GRAY2BGR)

# overlay
overlay = cv2.addWeighted(orig, 0.6, heatmap, 0.4, 0)

import matplotlib.pyplot as plt
plt.figure(figsize=(5,5))
plt.imshow(overlay[:,:,::-1])
plt.axis("off")
plt.title("AI Attention Heatmap")
plt.show()


In [ ]:
import numpy as np
import cv2
import torch

def generate_gradcam_pp(model, img_tensor):
    model.eval()

    gradients = []
    activations = []

    target_layer = model.features[-1]

    def forward_hook(module, inp, out):
        activations.append(out)

    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0])

    h1 = target_layer.register_forward_hook(forward_hook)
    h2 = target_layer.register_backward_hook(backward_hook)

    output = model(img_tensor)
    pred_class = output.argmax()

    model.zero_grad()
    output[0, pred_class].backward(retain_graph=True)

    grads = gradients[0].detach().cpu().numpy()[0]
    acts = activations[0].detach().cpu().numpy()[0]

    # Grad-CAM++ weighting
    alpha = grads**2
    alpha = alpha / (2 * grads**2 + np.sum(acts * grads**3, axis=(1,2), keepdims=True) + 1e-7)
    weights = np.sum(alpha * np.maximum(grads, 0), axis=(1,2))

    cam = np.zeros(acts.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * acts[i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (img_tensor.shape[3], img_tensor.shape[2]))
    cam = cam - cam.min()
    cam = cam / (cam.max() + 1e-8)

    h1.remove()
    h2.remove()

    return cam


In [ ]:
cam = generate_gradcam_pp(model, img_t)

heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)

orig = np.array(img.convert("L").resize((224,224)))
orig = cv2.cvtColor(orig, cv2.COLOR_GRAY2BGR)

overlay = cv2.addWeighted(orig, 0.65, heatmap, 0.35, 0)

import matplotlib.pyplot as plt
plt.figure(figsize=(5,5))
plt.imshow(overlay[:,:,::-1])
plt.axis("off")
plt.title("Medical Grad-CAM++ Heatmap")
plt.show()


In [ ]:
def generate_real_clinical_report(scores, context="Chest X-ray study"):
    top = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]

    primary = top[0][0]
    conf = top[0][1]

    findings = []

    if primary.lower() == "no finding":
        findings.append("The lungs are clear without focal consolidation, pleural effusion, or pneumothorax.")
        findings.append("Cardiomediastinal silhouette is within normal limits.")
    else:
        findings.append(f"Radiographic features suggestive of {primary.lower()} are observed.")
        findings.append("No evidence of acute life-threatening cardiopulmonary abnormality.")

    findings.append("Visualized osseous structures are intact.")
    findings.append("No significant interval change compared to prior study (if available).")

    impression = [
        f"Findings most consistent with {primary.lower()} (confidence {conf:.2f}).",
        "Clinical correlation recommended.",
        "Follow-up imaging may be considered based on symptoms."
    ]

    report = "\n".join([
        "FINDINGS:",
        *findings,
        "",
        "IMPRESSION:",
        *impression
    ])

    return report


In [ ]:
report = generate_real_clinical_report(scores)
print(report)


In [ ]:
def mix_mlp_reasoning(scores):
    refined = scores.copy()

    # --- Clinical co-occurrence rules ---

    # Heart failure pattern
    if refined.get("Edema", 0) > 0.35 and refined.get("Cardiomegaly", 0) > 0.30:
        refined["Heart Failure Pattern"] = max(refined.get("Heart Failure Pattern", 0), 0.85)
        refined["Edema"] += 0.05
        refined["Cardiomegaly"] += 0.05

    # Infectious pneumonia pattern
    if refined.get("Consolidation", 0) > 0.30 and refined.get("Pneumonia", 0) > 0.30:
        refined["Infectious Pattern"] = 0.80
        refined["Pneumonia"] += 0.05

    # Pleural disease
    if refined.get("Pleural Effusion", 0) > 0.35:
        refined["Pleural Disease Pattern"] = 0.75
        refined["Atelectasis"] = max(refined.get("Atelectasis", 0), 0.30)

    # Lung collapse hint
    if refined.get("Atelectasis", 0) > 0.40:
        refined["Volume Loss"] = 0.65

    # --- Normalize safely ---
    total = sum(refined.values()) + 1e-8
    for k in refined:
        refined[k] = refined[k] / total

    return refined


In [ ]:
scores_refined = mix_mlp_reasoning(scores)


In [ ]:
print("Top BEFORE reasoning:")
for k, v in list(scores.items())[:6]:
    print(f"{k:25s}: {v:.3f}")

print("\nTop AFTER reasoning:")
top_ref = sorted(scores_refined.items(), key=lambda x: x[1], reverse=True)
for k, v in top_ref[:6]:
    print(f"{k:25s}: {v:.3f}")


In [ ]:
report = generate_real_clinical_report(scores_refined)
print(report)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ProFAEncoder(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone.features   # DenseNet feature extractor

        # Projection heads for 3 levels
        self.pixel_proj = nn.Conv2d(1024, 256, kernel_size=1)
        self.region_proj = nn.Conv2d(1024, 256, kernel_size=1)
        self.organ_proj = nn.Conv2d(1024, 256, kernel_size=1)

    def forward(self, x):
        feat = self.backbone(x)   # shape: (B,1024,H,W)

        # Pixel-level (high resolution)
        pixel_feat = self.pixel_proj(feat)

        # Region-level (medium resolution)
        region_feat = F.avg_pool2d(feat, kernel_size=2)
        region_feat = self.region_proj(region_feat)

        # Organ-level (global)
        organ_feat = F.adaptive_avg_pool2d(feat, (1,1))
        organ_feat = self.organ_proj(organ_feat)

        return pixel_feat, region_feat, organ_feat


In [ ]:
profa = ProFAEncoder(model).eval()


In [ ]:
with torch.no_grad():
    pixel_feat, region_feat, organ_feat = profa(img_t)

print("Pixel-level feature:", pixel_feat.shape)
print("Region-level feature:", region_feat.shape)
print("Organ-level feature:", organ_feat.shape)


In [ ]:
def fuse_profa(pixel, region, organ):
    pixel_vec = pixel.mean(dim=[2,3])
    region_vec = region.mean(dim=[2,3])
    organ_vec = organ.view(organ.size(0), -1)

    fused = torch.cat([pixel_vec, region_vec, organ_vec], dim=1)
    return fused

fused_feature = fuse_profa(pixel_feat, region_feat, organ_feat)
print("Fused cognitive feature:", fused_feature.shape)


In [ ]:
import torch.nn as nn
import torch

class RCTA(nn.Module):
    def __init__(self, feat_dim, label_dim):
        super().__init__()

        # Attention layers
        self.image_to_context = nn.Linear(feat_dim, feat_dim)
        self.context_to_label = nn.Linear(feat_dim, label_dim)
        self.label_to_image = nn.Linear(label_dim, feat_dim)

        self.activation = nn.ReLU()

    def forward(self, img_feat, context_vec, label_vec):

        # Stage 1: Image queries Context
        ctx_attention = self.activation(self.image_to_context(img_feat))
        ctx_out = ctx_attention + context_vec

        # Stage 2: Context queries Labels (Hypothesis)
        label_attention = self.activation(self.context_to_label(ctx_out))
        label_out = label_attention + label_vec

        # Stage 3: Labels verify Image (Closed loop)
        img_attention = self.activation(self.label_to_image(label_out))
        verified_img = img_attention + img_feat

        return verified_img, label_out


In [ ]:
# Simple simulated context embedding
context_vec = torch.randn(1, fused_feature.shape[1])


In [ ]:
label_values = list(scores_refined.values())[:len(diseases)]
label_vec = torch.tensor(label_values).unsqueeze(0).float()


In [ ]:
rcta = RCTA(feat_dim=fused_feature.shape[1], label_dim=label_vec.shape[1]).eval()

with torch.no_grad():
    verified_img_feat, updated_labels = rcta(fused_feature, context_vec, label_vec)

print("Verified image feature:", verified_img_feat.shape)
print("Updated label hypothesis:", updated_labels.shape)


In [ ]:
def cognitive_radiology_inference(img, model, profa, rcta):
    # ---------------------------
    # 1. Preprocess image
    # ---------------------------
    import numpy as np
    import torchxrayvision as xrv
    import torch

    img_np = np.array(img.convert("L"))
    img_np = xrv.datasets.normalize(img_np, 255)
    img_t = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).float()
    img_t = torch.nn.functional.interpolate(img_t, size=(224,224), mode="bilinear", align_corners=False)

    # ---------------------------
    # 2. Base prediction
    # ---------------------------
    with torch.no_grad():
        preds = model(img_t)

    base_scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}

    # ---------------------------
    # 3. MIX-MLP reasoning
    # ---------------------------
    refined_scores = mix_mlp_reasoning(base_scores)

    # ---------------------------
    # 4. PRO-FA hierarchical vision
    # ---------------------------
    with torch.no_grad():
        pixel_feat, region_feat, organ_feat = profa(img_t)

    fused_feature = fuse_profa(pixel_feat, region_feat, organ_feat)

    # ---------------------------
    # 5. Create context + label vectors
    # ---------------------------
    context_vec = torch.randn(1, fused_feature.shape[1])

    label_values = list(refined_scores.values())[:len(diseases)]
    label_vec = torch.tensor(label_values).unsqueeze(0).float()

    # ---------------------------
    # 6. RCTA Cognitive Loop
    # ---------------------------
    with torch.no_grad():
        verified_img_feat, updated_labels = rcta(fused_feature, context_vec, label_vec)

    # ---------------------------
    # 7. Generate Clinical Report
    # ---------------------------
    report = generate_real_clinical_report(refined_scores)

    return refined_scores, report, img_t


In [ ]:
scores_final, final_report, img_t = cognitive_radiology_inference(img, model, profa, rcta)

print("FINAL REPORT:\n")
print(final_report)


In [ ]:
cam = generate_gradcam_pp(model, img_t)

heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)

orig = np.array(img.convert("L").resize((224,224)))
orig = cv2.cvtColor(orig, cv2.COLOR_GRAY2BGR)

overlay = cv2.addWeighted(orig, 0.65, heatmap, 0.35, 0)

import matplotlib.pyplot as plt
plt.figure(figsize=(5,5))
plt.imshow(overlay[:,:,::-1])
plt.axis("off")
plt.title("Cognitive AI Attention")
plt.show()


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import torch
import torchxrayvision as xrv
from PIL import Image
import cv2
import matplotlib.pyplot as plt

st.set_page_config(page_title="Cognitive Radiology AI", layout="wide")

# ------------------ STYLING ------------------
st.markdown("""
<style>
body {background-color: #0e1117;}
h1, h2, h3 {color: #e6edf3;}
.report-box {
    background-color: #161b22;
    padding: 20px;
    border-radius: 10px;
    border: 1px solid #30363d;
    color: #c9d1d9;
}
.metric-box {
    background-color: #161b22;
    padding: 10px;
    border-radius: 8px;
    border: 1px solid #30363d;
}
</style>
""", unsafe_allow_html=True)

st.title("🧠 Cognitive Radiology AI")
st.caption("Hierarchical Vision • Clinical Reasoning • Cognitive Attention")

# ------------------ LOAD MODEL ------------------
@st.cache_resource
def load_model():
    model = xrv.models.DenseNet(weights="densenet121-res224-all")
    model.eval()
    return model

model = load_model()
diseases = xrv.datasets.default_pathologies

# ------------------ UPLOAD ------------------
uploaded = st.file_uploader("Upload Chest X-ray", type=["png","jpg","jpeg"])

if uploaded:
    col1, col2 = st.columns([1,1])

    img = Image.open(uploaded).convert("RGB")
    col1.image(img, caption="Input X-ray", use_column_width=True)

    # -------- PREPROCESS --------
    img_np = np.array(img.convert("L"))
    img_np = xrv.datasets.normalize(img_np, 255)
    img_t = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).float()
    img_t = torch.nn.functional.interpolate(img_t, size=(224,224))

    # -------- PREDICT --------
    with torch.no_grad():
        preds = model(img_t)

    scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
    scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

    # ------------------ HEATMAP ------------------
    st.subheader("AI Visual Attention")

    gradients = []
    activations = []

    target_layer = model.features[-1]

    def f_hook(module, inp, out): activations.append(out)
    def b_hook(module, gin, gout): gradients.append(gout[0])

    h1 = target_layer.register_forward_hook(f_hook)
    h2 = target_layer.register_backward_hook(b_hook)

    output = model(img_t)
    cls = output.argmax()
    model.zero_grad()
    output[0, cls].backward()

    grads = gradients[0].detach().cpu().numpy()[0]
    acts = activations[0].detach().cpu().numpy()[0]

    weights = grads.mean(axis=(1,2))
    cam = np.zeros(acts.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * acts[i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (224,224))
    cam = cam / (cam.max()+1e-8)

    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    base = np.array(img.convert("L").resize((224,224)))
    base = cv2.cvtColor(base, cv2.COLOR_GRAY2BGR)
    overlay = cv2.addWeighted(base, 0.65, heatmap, 0.35, 0)

    col2.image(overlay[:,:,::-1], caption="Model Attention")

    # ------------------ METRICS ------------------
    st.subheader("Top Clinical Predictions")

    cols = st.columns(3)
    for i, (k,v) in enumerate(list(scores.items())[:6]):
        cols[i%3].metric(k, f"{v:.3f}")

    # ------------------ REPORT ------------------
    primary = list(scores.items())[0]

    report = f"""
FINDINGS:
Radiographic appearance suggests **{primary[0]}**.
No acute life-threatening cardiopulmonary abnormality identified.
Cardiomediastinal silhouette within acceptable limits.

IMPRESSION:
Findings most consistent with **{primary[0]}** (confidence {primary[1]:.2f}).
Clinical correlation recommended.
"""

    st.subheader("AI Generated Radiology Report")
    st.markdown(f"<div class='report-box'>{report}</div>", unsafe_allow_html=True)


In [ ]:
!pip install streamlit torch torchvision pillow torchxrayvision matplotlib opencv-python pyngrok


In [ ]:
from pyngrok import ngrok
import subprocess, time

process = subprocess.Popen(["streamlit","run","app.py","--server.port=8501"])
time.sleep(5)
print(ngrok.connect(8501))


In [ ]:
!npm install -g localtunnel


In [ ]:
import subprocess, time

process = subprocess.Popen(["streamlit","run","app.py","--server.port=8501"])
time.sleep(5)


In [ ]:
!lt --port 8501


In [ ]:
!curl ipv4.icanhazip.com


In [ ]:
!pip install -q gradio torch torchvision pillow torchxrayvision matplotlib opencv-python


In [ ]:
import gradio as gr
import numpy as np
import torch
import torchxrayvision as xrv
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# ---------- Load Model ----------
model = xrv.models.DenseNet(weights="densenet121-res224-all")
model.eval()
diseases = xrv.datasets.default_pathologies


# ---------- Heatmap ----------
def generate_heatmap(img_t):
    gradients = []
    activations = []

    target_layer = model.features[-1]

    def f_hook(module, inp, out):
        activations.append(out)

    def b_hook(module, gin, gout):
        gradients.append(gout[0])

    h1 = target_layer.register_forward_hook(f_hook)
    h2 = target_layer.register_backward_hook(b_hook)

    output = model(img_t)
    cls = output.argmax()
    model.zero_grad()
    output[0, cls].backward()

    grads = gradients[0].detach().cpu().numpy()[0]
    acts = activations[0].detach().cpu().numpy()[0]

    weights = grads.mean(axis=(1,2))
    cam = np.zeros(acts.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * acts[i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (224,224))
    cam = cam / (cam.max()+1e-8)

    h1.remove()
    h2.remove()
    return cam


# ---------- Main Function ----------
def analyze_xray(image):

    img = image.convert("L")
    img_np = np.array(img)
    img_np = xrv.datasets.normalize(img_np, 255)
    img_t = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).float()
    img_t = torch.nn.functional.interpolate(img_t, size=(224,224))

    with torch.no_grad():
        preds = model(img_t)

    scores = {diseases[i]: float(preds[0][i]) for i in range(len(diseases))}
    scores = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

    # Heatmap
    cam = generate_heatmap(img_t)
    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)

    base = np.array(img.resize((224,224)))
    base = cv2.cvtColor(base, cv2.COLOR_GRAY2BGR)
    overlay = cv2.addWeighted(base, 0.65, heatmap, 0.35, 0)

    # Report
    primary = list(scores.items())[0]
    report = f"""
FINDINGS:
Radiographic appearance suggests {primary[0]}.
No acute life-threatening abnormality detected.

IMPRESSION:
Most consistent with {primary[0]} (confidence {primary[1]:.2f}).
Clinical correlation recommended.
"""

    top_text = "\n".join([f"{k}: {v:.3f}" for k,v in list(scores.items())[:6]])

    return overlay[:,:,::-1], top_text, report


# ---------- UI ----------
demo = gr.Interface(
    fn=analyze_xray,
    inputs=gr.Image(type="pil", label="Upload Chest X-ray"),
    outputs=[
        gr.Image(label="AI Attention Heatmap"),
        gr.Textbox(label="Top Predictions"),
        gr.Textbox(label="Radiology Report")
    ],
    title="🧠 Cognitive Radiology AI",
    description="Hierarchical Vision • Clinical Reasoning • Cognitive Attention",
)

demo.launch(share=True)


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import matplotlib.pyplot as plt

# -------- Grad-CAM --------
def generate_gradcam(model, img_tensor, target_index=0):
    model.eval()

    gradients = []
    activations = []

    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0])

    def forward_hook(module, inp, out):
        activations.append(out)

    # Hook last conv layer
    target_layer = model.features[-1]
    handle_f = target_layer.register_forward_hook(forward_hook)
    handle_b = target_layer.register_backward_hook(backward_hook)

    output = model(img_tensor)
    score = output[0, target_index]
    model.zero_grad()
    score.backward()

    grads = gradients[0]
    acts = activations[0]

    weights = torch.mean(grads, dim=(2, 3), keepdim=True)
    cam = torch.sum(weights * acts, dim=1).squeeze()
    cam = F.relu(cam)

    cam = cam.detach().cpu().numpy()
    cam = cv2.resize(cam, (224, 224))
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

    handle_f.remove()
    handle_b.remove()

    return cam


In [ ]:
def show_heatmap_on_image(img, cam):
    img_np = np.array(img.resize((224, 224)).convert("RGB"))

    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img_np, 0.6, heatmap, 0.4, 0)

    plt.figure(figsize=(6,6))
    plt.imshow(overlay)
    plt.axis("off")
    plt.title("AI Attention Heatmap (Grad-CAM)")
    plt.show()


In [ ]:
top_disease_index = np.argmax(preds.detach().cpu().numpy())
cam = generate_gradcam(model, img_t, top_disease_index)
show_heatmap_on_image(img, cam)


In [ ]:
import torch
torch.save(model.state_dict(), "model.pth")
